In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

# Gamma rhythms (30-80 Hz)

## Getting Started

In order to understand the workflow and initial parameter sets provided with this tutorial, we must first briefly describe prior studies on the mechanistic origin of gamma rhythms, including our prior modeling work that led to the creation of the parameter sets you will work with [1].

Gamma rhythms can encompass a wide band of frequencies from 30-150 Hz. Here, we will focus on the generation of so called low gamma rhythms in the 30-80 Hz range. It has been well established through experiments and computational modeling that these rhythms can emerge in local spiking networks through interactions of excitatory cell and inhibitory cell interactions, with the period of the oscillation set by the time constant of decay of GABAA-mediated inhibitory currents [2–4], a mechanism that has been referred to as pyramidal-interneuronal gamma (PING). In normal regimes, the decay time constant of GABAA-mediated synapses (around 25 ms) bounds oscillations to the low gamma frequency band (around 40 Hz).

In general, PING rhythms are initiated by “excitation” to the excitatory (E) cells that causes spiking, which in turn synaptically activates a spiking population of inhibitory (I) cells. In turn, these I cells inhibit the E cells, preventing further E cell activity until the E cells can overcome the effects of the inhibition (~25 ms later). The pattern is repeated, creating a gamma frequency oscillation (around 40 Hz, 40 spikes/second). This general principle is schematically described in Figure 1 below. The frequency of the rhythm is paced by this time constant of decay of inhibition, which is mediated by strong GABA-A currents, as well as the excitability of the E cells (if the E cells are very excitable, they can fire before the inhibition has completely worn off, and the oscillation will be faster).


<img src="https://raw.githubusercontent.com/jonescompneurolab/hnn-tutorials/master/gamma/images/image19.png" width="500"></img>
<center>Schematic illustration of the circuit mechanisms underlying PING rhythms.</center>

In this tutorial, we will explore the generation of PING rhythms in the HNN model. We will provide example parameter files and walk through simulations that generate low-frequency gamma activity in both Layers 2/3 and Layer 5, as in [1]. This tutorial relies on a different type of exogenous drive to “activate” the local network than the other tutorials. Here, the necessary excitation to generate spiking in the pyramidal neurons (E cells) that initiates the rhythm (see PING description above) is provided by a continuous train of action potentials with a Poisson distribution (Poisson Input) that activate post-synaptic excitatory AMPA synapses on the pyramidal neurons. This Poisson drive causes the pyramidal neurons to fire, dependent on the chosen conductance of the AMPA currents. The inhibition in the network is strong enough to overcome the Poisson drive and entrain the network spiking into a gamma frequency rhythm.

Of note, gamma rhythms can be generated by circuit mechanisms other than PING, including subthreshold rhythmic exogenous drive. Lee and Jones (2013) examined various mechanisms of generation of gamma activity and described ways to distinguish the mechanisms of generation based on features of current dipole signal [1]. After completing this tutorial, we encourage you to explore alternate mechanisms of gamma generation and to compare to your own gamma data.

In [ ]:
import time
from hnn_core.gui.gui import HNNGUI
from ipywidgets import Output
from IPython.display import Javascript

# do not mix this with GUI operations
gui = HNNGUI()

In [ ]:
# In practice, set return_layout to false to show the dashboard

# gui.compose(return_layout=False)

from IPython.display import display
app_layout = gui.compose()
display(app_layout)

In [ ]:
gui.capture()

<!-- We ignored the load data part for now -->

##   Load/view parameters to define network structure & to “activate” the network

An initial parameter set that will simulate PING rhythms as described in the Getting Started section above can be downloaded via the following hyperlink: [gamma_L5weak_L2weak.param](https://github.com/jonescompneurolab/hnn/blob/master/param/gamma_L5weak_L2weak.param). This file can also be found in the HNN param subfolder.

The template cortical column networks structure for this simulation is described in the Overview and What’s Under the Hood sections of the HNN webpage. Several of the network parameter (i.e., local excitatory and inhibitory connection strengths) will be adjusted in this tutorial, but we will begin by describing the inputs the “activate” the network.

To load the initial parameter set, navigate to the HNN GUI and click:

    Load network

Once you have this param file downloaded and existing drive deleted, upload the file to the GUI through the `Load network` button.

In [ ]:
param_url = "https://raw.githubusercontent.com/jonescompneurolab/hnn/master/param/gamma_L5weak_L2weak.param"
gui._simulate_upload_connectivity(param_url)
gui._simulate_upload_drives(param_url)

In this simulation, the network will be “activated” with excitatory AMPA synaptic input, distributed in time as a Poisson process and given to somas of the pyramidal neurons in Layer 2/3 and Layer 5. This noisy input will generate spiking activity in the pyramidal neurons that can initiate the PING rhythm.

To view the parameters that “activate” the network and initiate a gamma rhythm, return to the main GUI window and click:

    External drives

In [ ]:
gui._simulate_left_tab_click("External drives")

In [ ]:
gui.capture()

In this parameter file, all synaptic connections within the network are turned off (weight = 0 ), except for reciprocal connections between the excitatory (AMPA only) and inhibitory (GABAA only) cells within the same layer. In other words, the Layer 2/3 and Layer 5 networks are not connected to each other (see the Under the Hood section of the HNN website for further details on connectivity structure). This is not biologically realistic, but was done for illustration purposes and to prevent pyramidal-to-pyramidal interactions from disrupting the gamma rhythm.

To view the local network connection parameters, click::

    Network connectivity
    
You should see the values of adjustable parameters displayed as in the dialog boxes below. Notice that the Layer 2/3 and Layer 5 are not connected to each other, the inhibitory conductance weights within layers are stronger than the excitatory conductances, and there are also strong inhibitory-to-inhibitory connections (i.e., basket-to-basket). This strong autonomous inhibition will cause synchrony among the basket cells, and hence strong inhibition onto the pyramidal neurons (note: this synapse was not depicted in Figure 1).

In [ ]:
gui._simulate_left_tab_click("Network connectivity")

In [ ]:
gui.capture()

## 2. Run the simulation and visualize net current dipole

To run this simulation, click:

    Run

In [ ]:
gui.run_button.click()

In [ ]:
gui.capture()

In this simulation, a single histogram displaying the Poisson drive to the excitatory cells is shown in the top panel, displaying no clear rhythmicity at the gamma frequency. The Poisson drive provided causes pyramidal cells to fire, which in-turn cause the inhibitory neurons to fire, and feedback inhibition from the interneurons to the pyramidal cells generates a regular gamma rhythm via the PING mechanism described above (the process to view the spiking activity is described in Section 3 below). In this example, the rhythmic dipole displays sharp downward deflections, reflective of the strong inhibition onto the pyramidal neuron somas, which pulls current flow down the dendrites. The corresponding spectrogram confirms that, for this parameter set, the dipole signal contains strong oscillatory components in the gamma range (~50 Hz).

## 3. Calculating and Viewing Spiking Activity and Power Spectral Density (PSD)

To view the spiking activity generated by the different neuron populations in the network, select "spikes" from the dropdown menu in the visualization window:

In [ ]:
gui._simulate_viz_action("edit_figure", "Figure 1", "ax1", "default", "current dipole", {}, "clear")
gui._simulate_viz_action("edit_figure", "Figure 1", "ax1", "default", "spikes", {}, "plot")

In [ ]:
gui.capture()

Notice that the excitatory pyramidal neurons in each layer (green and red dots / lines) fire before the inhibitory basket cells in each layer (white, blue dots / lines). The pyramidal neuron firing drives the basket cells to fire. The basket cells are highly synchronous due to the strong inhibitory-to-inhibitory connections. The basket cells then prevent the pyramidal neurons from firing for ~25 ms, generating the PING rhythms. The line-plots, which show spike counts over time, also demonstrate rhythmicity. The pyramidal neurons are firing periodically, but with lower synchrony due to the Poisson drive (orange histogram at top), which creates randomized spike times across the populations (once the inhibition sufficiently wears off). This type of dispersed pyramidal neuron firing is considered “weak” PING - hence the parameter file name including weak.

We can further confirm that this oscillation is in the gamma frequency range by viewing the power spectral density of the current dipole signal (average spectrogram across time).

In [ ]:
gui._simulate_viz_action("switch_fig_template", "single figure")
gui._simulate_viz_action("add_fig")
gui._simulate_viz_action("edit_figure", "Figure 2", "ax0", "default", "PSD", {}, "plot")

In [ ]:
gui.capture()

Notice that the power in the gamma band is much smaller in Layer 2/3 than in Layer 5. This is reflective, in part, of the fact that the length of the Layer 2/3 pyramidal neurons is smaller than Layer 5, and hence Layer 2/3 cells produce smaller current dipole moments that can be masked by activity in Layer 5 (see [1] for further discussion).

## 4. Adjusting parameters

In the following simulations, we explore parameter alterations influencing the generation of gamma rhythms.

Also note that before running/loading new simulations, we first remove the prior simulation by pressing the “Remove Simulation” button at the bottom of the GUI. If we do not do this, both simulation dipoles are displayed (old simulation with dotted line, new simulation with solid line; see “Tour of the GUI” for more details on simulation control).

### 4.1 Two key parameters controlling gamma rhythmicity are cell “excitability” and network connectivity

In this exercise, HNN will be used to explore the impact of changing several key parameters that control gamma rhythmicity in networks, including cell excitabilityand network connectivity. Cell “excitability” can be adjusted in many ways reflecting the intrinsic properties of a neuron (e.g. membrane resting potential) and/or the influence of external factors such as noisy background inputs or tonic drive via neuromodulation. Parameters controlling network connectivity include the synaptic connection strengths between cells and/or the time constants of synaptic activation. Here, we explore the influence of a few of these factors on gamma rhythmicity. To do so, we will examine activity in only one of the layers, namely Layer 5, by setting all inputs and connectivity in Layer 2/3 to zero.

First, reload the parameter file gamma_L5weak_L2weak.param (see step 1 above). To turn off all off the Layer 2/3 activity, load the network parameters with:

    Load network

Adjust parameters so that all Layer 2/3 weights are set to 0.0 as shown in the dialog boxes below.

In [ ]:
for key, title in gui._conn_drive_s['connectivity']._titles.items():
    soure_targets = title.split(" ")[0].split("→")
    l2inst = all(['L2' in st for st in soure_targets])
    if l2inst is True:
        for widget_combined in gui._conn_drive_s['connectivity'].children[int(key)].children:
            for widget in widget_combined.children:
                if type(widget.value) in (float, int):
                    widget.value = 0

In [ ]:
gui._simulate_left_tab_click("Network connectivity")

In [ ]:
gui.capture()

Next, load the Poisson input parameters by selecting:

    Drives > Poisson

In [ ]:
gui._simulate_left_tab_click("External drives")

In [ ]:
gui.capture()

Adjust the strength of the Layer 2/3 Pyr AMPA weight to 0.0 as shown below.

In [ ]:
for key, title in gui._conn_drive_s['drive']._titles.items():
    if title.split(" ")[0] == 'extpois':
        widgets = gui._conn_drive_s['drive'].children[int(key)].children
        for widget in widgets:
            if type(widget.value) in (float, int) and 'L2' in widget.description:
                widget.value = 0

In [ ]:
gui.widget_simulation_name.value = 'default2'

Save the simulation with a new “Simulation Name” in the Set Parameters dialog box: “gamma_L5weak_only” and run the simulation by clicking Start Simulation in the main HNN GUI window. The main HNN GUI will show a gamma rhythm similar to that observed in Step 2, that looks as follows:

In [ ]:
gui.run_button.click()

In [ ]:
gui.capture()

However, if you now view the Simulation Spiking Activity and PSD (see step 3 above) you will see activity from only Layer 5 that looks as follows.

In [ ]:
gui._simulate_switch_plot_type(0, "spikes")
gui._simulate_switch_plot_type(1, "PSD")

In [ ]:
gui.capture()

Notice the weak PING rhythm in Layer 5 consisting of weakly synchronous pyramidal neuron firing, followed by synchronous inhibitory neuron firing, that gates the network dipole rhythm to ~50 Hz.

To examine the impact of cell excitability on gamma expression, we can increase the excitability of the Layer 5 pyramidal neurons by adding a tonic applied current that might represent a neuromodulatory influence. Load the Tonic Input parameters with Set Parameters> Tonic Inputs.

As in the Tonic Inputs dialog box below, click on Layer 5, and adjust the amplitude of the tonic drive to the L5 Pyr to be 6.0 nA. This level of injected current will be applied to the soma of the L5 Pyramidal neurons starting at time 0.0 ms and remaining on throughout the simulation (stop time = -1.0 ms).

In [ ]:
# how to show this?

Save the simulation with a new simulation name (e.g., “gamma_L5weak_only_tonic”) in the Set Parameters dialog box and run the simulation by clicking Start Simulation in the main HNN GUI window. The simulation will yield the following output.

In [ ]:
gui.run_button.click()

In [ ]:
gui.capture()

Notice that the oscillation waveform is now more regular with less noise due to the fact that the tonic drive is strong and outweighs the influence of the Poisson drive.

Notice that the Layer 5 pyramidal neurons are now firing nearly synchronously. They in turn synchronously activate the inhibitory basket neurons, which then inhibit the pyramidal neurons for ~20 ms, when the tonic drive outweighs the inhibition and the pyramidal neurons firing again creating a ~50 Hz PING rhythm. This type of synchronous rhythm is sometimes referred to as “strong” PING. Notice that there is also a ~90 Hz component to the dipole spectrogram. Inspection of the dipole waveform shows that this 90 Hz activity does not represent a separate rhythm, but rather reflects the dipole waveform shape which has a fast oscillation on each cycle of the slower rhythm, due to the strong downward currents induced by strong somatic inhibition on the pyramidal neurons.

### 4.1.1 Exercises for further exploration

- How else might the excitability of the cells be adjusted? What happens to the network dynamics? Can you make the oscillation faster or slower?

- How else might the synaptic connections in the network be adjusted to impact gamma? What happens to the network dynamics? Can you make the oscillation faster or slower?

- Gamma rhythms can be created in neworks where the pyramidal neurons do not fire on their own but their activity is still regulated by interneuron firing (interneuron mediated gamma: ING). Can you adjust parameters to create such an oscillation in the network?


## 5. Have fun exploring your own data!

We have not observed strong gamma activity in our SI dipole data, and as such have not provided an example data set to compare simulation results to, as in the other tutorials. However, simulation results can be compared to recorded data, as described in the ERP and Alpha/Beta tutorials. Follow steps 1-4 above using your data and parameter adjustments based on your own hypotheses.


## References

- Lee, S. & Jones, S. R. Distinguishing mechanisms of gamma frequency oscillations in human current source signals using a computational model of a laminar neocortical network. Front. Hum. Neurosci. 7, 869 (2013).

- Cardin, J. A. et al. Driving fast-spiking cells induces gamma rhythm and controls sensory responses. Nature 459, 663–667 (2009).

- Vierling-Claassen, D., Cardin, J. A., Moore, C. I. & Jones, S. R. Computational modeling of distinct neocortical oscillations driven by cell-type selective optogenetic drive: separable resonant circuits controlled by low-threshold spiking and fast-spiking interneurons. Front. Hum. Neurosci. 4, 198 (2010).

- Buzsáki, G. & Wang, X.-J. Mechanisms of gamma oscillations. Annu. Rev. Neurosci. 35, 203–225 (2012).
